<a href="https://colab.research.google.com/github/soprunveronika/DSL/blob/main/Job3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def form_all_firsts(G: dict) -> dict:
  result_dict = {}  # будет заполнен и возвращен 
  tokens = G['toks']
  variables = G['vars']

  def first(non_terminal: str) -> set:
    definition = variables[non_terminal]
    answer_set = set()  # набор результатов первой функции 
    has_epsilon = False
    for rule in definition:
      if not len(rule):     # случай, когда нетерминал истекает 
        has_epsilon = True
        continue
      for index, symbol in enumerate(rule):
        if symbol in tokens:  # случай, когда первый символ является терминальным 
          answer_set.add(symbol)
          break
        else:                 # случай, когда первый символ является нетерминальным 
          if symbol in result_dict:  # нам не нужно рассчитывать FIRST, если он уже рассчитан 
            firsts_for_another = result_dict[symbol]
            answer_set = set.union(answer_set, firsts_for_another)
          else: # вычисление, кеширование и добавление результата FIRST функции 
            firsts_for_another = first(symbol)
            result_dict[symbol] = firsts_for_another
            answer_set = set.union(answer_set, firsts_for_another)
          if '' not in firsts_for_another:
            break
          elif index == len(rule) - 1:
            has_epsilon = True
    
    if has_epsilon:
      answer_set.add('')
    elif '' in answer_set:
      answer_set.remove('')
    
    return answer_set

  for non_terminal in variables.keys():
    # рассчитываем, только если он еще не рассчитан внутри вызова FIRST
    if non_terminal not in result_dict:
      result_dict[non_terminal] = first(non_terminal)

  return result_dict

Функция FIRST возвращает набор терминалов, которые могут быть вычтены из нетерминала, который мы ей передали. Используя тот факт, что мы удалили левую рекурсию из нашей грамматики, мы можем легко реализовать эту функцию рекурсивно, потому что есть свидетельства того, что мы не застрянем в бесконечном цикле. Итак, если у нас есть нетерминальный A, мы начинаем перебирать правила, в которых левый сайт имеет только A, и если первый символ правила является терминальным h - мы просто добавляем его в set. Если нетерминал B - получаем ПЕРВЫЙ (B). Если он содержит '', мы проделываем те же операции со следующим символом правила и добавляем FIRST (B) \ {''} к нашему набору ответов. Если мы дошли до конца правила и ПЕРВЫЙ (последний) содержит "", то ответ также будет содержать "".

Я  реализовываю функцию form_all_first, которая берет грамматику и возвращает dict, где все ключи не являются терминалами, а значения являются результатом применения к ним FIRST. 

In [ ]:
def first_for_sequence(first_dict: dict, seq: str, tokens: set):
  answer = set()  # набор результатов 
  for symbol in seq:
    if symbol in tokens:  # нет смысла продолжать 
      answer.add(symbol)
      return answer
    else:
      answer = set.union(answer, first_dict[symbol])
      if '' not in first_dict[symbol]:
        return answer
      else:
        answer.remove('') # epsilon еще не на первом месте 
  answer.add('')  # если все символы ->> epsilon, seq ->> epsilon
  return answer

def form_all_follow(G: dict) -> dict:
  result_dict = {}  # будет заполнен и возвращен
  tokens = G['toks']
  variables = G['vars']
  hvar = G['hvar']
  updated = True
  epsilon = ''
  first_dict = form_all_firsts(G)

  for non_terminal in variables.keys():
    result_dict[non_terminal] = set()

  result_dict[hvar] = set()
  result_dict[hvar].add('$')
  while updated:
    updated = False
    for non_terminal, definition in variables.items():
      for rule in definition:
        for index, symbol in enumerate(rule):
          if symbol not in tokens:
            first_set = first_for_sequence(first_dict, rule[index + 1:], tokens)
            prev_size = len(result_dict[symbol])
            if epsilon in first_set:
              first_set.remove(epsilon)
              result_dict[symbol] = set.union(result_dict[symbol], first_set)
              result_dict[symbol] = set.union(result_dict[symbol], result_dict[non_terminal])
            else:
              result_dict[symbol] = set.union(result_dict[symbol], first_set)
            new_size = len(result_dict[symbol])
            if new_size != prev_size:
              updated = True
  
  return result_dict

Эта функция FOLLOW. Алгоритм вычисляет все терминальные символы, которые могут появиться после нетерминальных в начальной дедукции. Он рассчитывается один раз и навсегда, поэтому наша функция будет принимать грамматику и возвращать dict для каждого нетерминала. Начиная, мы устанавливаем follow_dict [S] = {$}, где $ - конец ввода. Для каждого правила, если A -> aB, поместите follow (A) в follow (B). если A -> aBb, то поместите first (b) в follow (A), где first (seq) - это все терминалы, которые могут начинать слово, вычитаемое из seq. Мы повторяем эти действия в цикле, потому что следующие наборы могут изменяться на каждой итерации, поэтому мы повторяем это до тех пор, пока после передачи всех правил dict не останется неизменным. 